# Low Urban Historic
This indicator was used in South Atlantic 2021 Blueprint.

It uses data representing the National Register of Historic Places. There is a national database for this from the park service, but the data has not been maintained and the polygons have huge slivers that are clearly errors. We dont use the polygon data from the national dataset. We do use the point data from the national dataset, but only in places where we don't have better data. 

We have state level data representing the National Register of Historic Places for a handfull of states. These data are more regularly maintained. When we have state level data, we use it. 

Because we haven't had time to re-run this indicaor, but we have wanted to make changes (like legend color changes). Some of those changes were made by hand and then just added to this code to try to keep a record. Due to those changes, it is possible that this code won't run as is. We have tried to add the steps we did by hand to this code, but it is also possible that we missed some things.

In [1]:
import os
import arcpy

In [2]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
NSourceWorkspace = r"G:\GIS_DATA\Cultural_Resources\NationalRegisterOfHistoricPlaces\NRIS_CR_Standards_Public.gdb"
SSourceWorkspace = r"G:\GIS_DATA\Cultural_Resources\NationalRegisterOfHistoricPlacesStateData"
OutWorkspace = r"E:\WORKING\Indicators\Terrestrial\LowUrbanHistoric\2021\LowUrbanHistoric.gdb"

In [334]:
# define final outputs
SEout = r"E:\WORKING\BaseBlueprint\2022\4_Indicators\SouthAtlanticLowUrbanHistoric\SouthAtlanticLowUrbanHistoric11.tif"

In [340]:
OutWorkspace = r"E:\WORKING\BaseBlueprint\2022\4_Indicators\SouthAtlanticLowUrbanHistoric\SouthAtlanticLowUrbanHistoric.gdb"

In [322]:
# define sub-indicator outputs to help with user support

In [323]:
# define rasters used for cell size, extent, and snapping
SEraster= r"F:\GIS_DATA\SALCC_BLUEPRINT\Version_2021\Blueprint_2021_Data_Download_20220204\1_ExtentLayers\SA2021extent.tif"

In [6]:
# define additional inputs
# state boundaries
states= r"G:\GIS_DATA\Boundaries.gdb\PoliticalLargeScale\Counties_TigerLine_2015_State"
# A list of the state level point data to be used in this project 
SCpt='G:\\GIS_DATA\\Cultural_Resources\\NationalRegisterOfHistoricPlacesStateData\\StateVersions_NC_FL_SC_NationalRegister\\SC GIS Data_2019_0628_restrictedRemoved\\SC GIS Data_2019_0628\\NRHP Structures_Non-Restricted\\SC_NRHP_Structures_Non-Restricted.shp'
VApt= 'G:\\GIS_DATA\\Cultural_Resources\\NationalRegisterOfHistoricPlacesStateData\\VDHR_HDs_and_Listed.gdb\\Listed_Site_Points_Virginia'
# did not use all NC points, only points that seem to be from the national register based on their values in the YearNR field
# I created the NC input seperately b/c that dataset was acting super weird and I could only get my select to work in an map session
NCpt= 'G:\\GIS_DATA\\Cultural_Resources\\NationalRegisterOfHistoricPlacesStateData\\StateVersions_NC_FL_SC_NationalRegister\\NCHPO_GISdata_2019-06-03\\NCHPOpoints_YearNRnotblank.shp'
nlcd= r"G:\GIS_DATA\LanduseLandcover\NLCD\NLCD_Land_Cover_L48_20190424\NLCD_2016_Land_Cover_L48_20190424_SECASbuff_N83.img"

In [324]:
# define additional inputs
SubR= r"E:\WORKING\BaseBlueprint\2022\1_SubregionsAndExtent\BaseBlueprintSubRgn.shp"
SAraster= r"F:\GIS_DATA\SALCC_BLUEPRINT\Version_2021\Blueprint_2021_Data_Download_20220204\1_ExtentLayers\SA2021extent.tif"

## National Park Service Register of Historic Places
We use the national data for states where we don't have state level data.

### Merge all the points from the National Park Service Register of Historic Places

In [7]:
# Set the workspace where the most recent download of the register of historic places is
arcpy.env.workspace = NSourceWorkspace

In [8]:
print(arcpy.env.workspace)

G:\GIS_DATA\Cultural_Resources\NationalRegisterOfHistoricPlaces\NRIS_CR_Standards_Public.gdb


In [9]:
#look in the source data from the National Park Service Register of Historic Places, make a list of all the point 
# feature classes
FCList = []

walk = arcpy.da.Walk(NSourceWorkspace, datatype="FeatureClass", type="Point")

for dirpath, dirnames, filenames in walk:
    for filename in filenames:
        FCList.append(os.path.join(dirpath, filename))

In [10]:
#Print the list made in the above step to make sure it is correct
print(FCList)

['G:\\GIS_DATA\\Cultural_Resources\\NationalRegisterOfHistoricPlaces\\NRIS_CR_Standards_Public.gdb\\Historic_Buildings\\crbldg_pt', 'G:\\GIS_DATA\\Cultural_Resources\\NationalRegisterOfHistoricPlaces\\NRIS_CR_Standards_Public.gdb\\Historic_Sites\\crsite_pt', 'G:\\GIS_DATA\\Cultural_Resources\\NationalRegisterOfHistoricPlaces\\NRIS_CR_Standards_Public.gdb\\Historic_Districts\\crdist_pt', 'G:\\GIS_DATA\\Cultural_Resources\\NationalRegisterOfHistoricPlaces\\NRIS_CR_Standards_Public.gdb\\Historic_Objects\\crobj_pt', 'G:\\GIS_DATA\\Cultural_Resources\\NationalRegisterOfHistoricPlaces\\NRIS_CR_Standards_Public.gdb\\Historic_Structures\\crstru_pt']


In [341]:
# Change the workspace to where I am saving the outputs
arcpy.env.workspace = OutWorkspace

In [342]:
# Print the current workspace to make sure I'm in the right spot
print(arcpy.env.workspace)

E:\WORKING\BaseBlueprint\2022\4_Indicators\SouthAtlanticLowUrbanHistoric\SouthAtlanticLowUrbanHistoric.gdb


In [13]:
# Merge all the point feature classes from the National Park Service Register of Historic Places, using the list 
# created above as the input
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster):
    arcpy.management.Merge(FCList,'MergeNational', 'NO_SOURCE_INFO')

### Remove points in states where we have more accurate state level data (Virginia, North Carolina, Florida, South Carolina). Then buffer remaining points by 100 meters.

In [14]:
print(arcpy.env.workspace)

E:\WORKING\Indicators\Terrestrial\LowUrbanHistoric\2021\LowUrbanHistoric.gdb


In [15]:
# make a layer with all the states where we DO NOT have more accurate data
arcpy.analysis.Select(states, "StatesWOData", "STATEFP IN ('54','21','29','40','48','22','05','28','01','47','13','72','78')")

<Result 'E:\\WORKING\\Indicators\\Terrestrial\\LowUrbanHistoric\\2021\\LowUrbanHistoric.gdb\\StatesWOData'>

In [16]:
# clip points to states where we do not have more accurate data. 
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster):
    arcpy.analysis.Clip("MergeNational", "StatesWOData", "MergeNationalClip")

In [17]:
# buffer the remaining points by 100 meters
arcpy.analysis.Buffer("MergeNationalClip", "MergeNationalClipBuff", "100 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")

<Result 'E:\\WORKING\\Indicators\\Terrestrial\\LowUrbanHistoric\\2021\\LowUrbanHistoric.gdb\\MergeNationalClipBuff'>

## State Level Data for the National Register of Historic Place Data

This is a little confusing, because this is still the National Register of Historic Places, but the states have MUCH more accurate locations than is in the Park Service layer

### We start with the point data from the states

For VA, SC, and Florida we use all the points. For North Carolina we have to remove some state designated areas that are not really national historic places

There is something weird about the NC point shapefile. I can't interact with it much at all. I can't reproject it. I can't copy it into a geodatabase. I was able to use "select by attribute" on it to select points that had a blank in the YearNR field (YearNR <> ' '), but only in an arcGIS pro map session, not in the tool box or in notebooks. After selection, I was able to export it to a new shapefile. That is what I'm using as my input here.

In [18]:
# Set the workspace where I want to save my output
arcpy.env.workspace = OutWorkspace

In [19]:
print(arcpy.env.workspace)

E:\WORKING\Indicators\Terrestrial\LowUrbanHistoric\2021\LowUrbanHistoric.gdb


In [20]:
# Make a list of the state level point data to be used in this project 
StatePt = [SCpt, NCpt, VApt]

In [21]:
# Merge all the point feature classes from the State Level National Register of Historic Places, using the list 
# created above as the input
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster):
    arcpy.management.Merge(StatePt,'MergeStatePoint', 'NO_SOURCE_INFO')

In [22]:
# buffer the state points by 100 meters
arcpy.analysis.Buffer("MergeStatePoint", "MergeStatePointBuff", "100 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")

<Result 'E:\\WORKING\\Indicators\\Terrestrial\\LowUrbanHistoric\\2021\\LowUrbanHistoric.gdb\\MergeStatePointBuff'>

### Next we work with the polygon data from the states

We use all of the polygon layers that were provided.


In [23]:
# Set the workspace where the most recent download of the State level register of historic places is
arcpy.env.workspace = SSourceWorkspace

In [24]:
print(arcpy.env.workspace)

G:\GIS_DATA\Cultural_Resources\NationalRegisterOfHistoricPlacesStateData


In [25]:
#look in the source data from the State level National Park Service Register of Historic Places, make a list of all 
# the polygon feature classes
FCList2 = []

walk = arcpy.da.Walk(SSourceWorkspace, datatype="FeatureClass", type="Polygon")

for dirpath, dirnames, filenames in walk:
    for filename in filenames:
        FCList2.append(os.path.join(dirpath, filename))

In [26]:
print(FCList2)

['G:\\GIS_DATA\\Cultural_Resources\\NationalRegisterOfHistoricPlacesStateData\\StateVersions_NC_FL_SC_NationalRegister\\Janet Cakir NR GIS FL_june2019\\NationalRegister.shp', 'G:\\GIS_DATA\\Cultural_Resources\\NationalRegisterOfHistoricPlacesStateData\\StateVersions_NC_FL_SC_NationalRegister\\NCHPO_GISdata_2019-06-03\\Local_District_Boundaries.shp', 'G:\\GIS_DATA\\Cultural_Resources\\NationalRegisterOfHistoricPlacesStateData\\StateVersions_NC_FL_SC_NationalRegister\\NCHPO_GISdata_2019-06-03\\NCHPO_NR_SL_DOE_Boundaries.shp', 'G:\\GIS_DATA\\Cultural_Resources\\NationalRegisterOfHistoricPlacesStateData\\StateVersions_NC_FL_SC_NationalRegister\\SC GIS Data_2019_0628_restrictedRemoved\\SC GIS Data_2019_0628\\NRHP Districts and Areas_Non-Restricted\\SC_NRHP_Districts-Areas_Non-Restricted.shp', 'G:\\GIS_DATA\\Cultural_Resources\\NationalRegisterOfHistoricPlacesStateData\\VDHR_HDs_and_Listed.gdb\\Listed_Historic_District_Boundaries_Virginia']


In [27]:
# Change the workspace to where I am saving the outputs
arcpy.env.workspace = OutWorkspace

In [28]:
print(arcpy.env.workspace)

E:\WORKING\Indicators\Terrestrial\LowUrbanHistoric\2021\LowUrbanHistoric.gdb


In [29]:
# Merge all the polygon feature classes from the State level National Register of Historic Places, using the list 
# created above as the input
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster):
    arcpy.management.Merge(FCList2,'MergeStatePoly', 'NO_SOURCE_INFO')

In [30]:
# Buffer the merged polygon layer by 30 meters. We did this because some of the polygons are so small that they 
# weren't represented when we convereted to 30 meter rasters. The buffer allows them to be represented in the raster.
arcpy.analysis.Buffer("MergeStatePoly", "MergeStatePolyBuff", "30 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")

<Result 'E:\\WORKING\\Indicators\\Terrestrial\\LowUrbanHistoric\\2021\\LowUrbanHistoric.gdb\\MergeStatePolyBuff'>

## Make a raster of all historic places

Now we merge together the 3 different layers we made above: buffered national points for states we don't have data for, buffered state points, buffered state polygons

In [31]:
# Merge 3 processed inputs we made above
arcpy.management.Merge(["MergeStatePointBuff","MergeStatePolyBuff","MergeNationalClipBuff"], "MergeAllBuffered", "NO_SOURCE_INFO")

<Result 'E:\\WORKING\\Indicators\\Terrestrial\\LowUrbanHistoric\\2021\\LowUrbanHistoric.gdb\\MergeAllBuffered'>

In [32]:
# add and calculate a field that we can use to convert to a raster
arcpy.management.CalculateField("MergeAllBuffered", "raster", "1", "PYTHON3", '', "TEXT")

<Result 'E:\\WORKING\\Indicators\\Terrestrial\\LowUrbanHistoric\\2021\\LowUrbanHistoric.gdb\\MergeAllBuffered'>

In [33]:
# convert all historic polygon to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    arcpy.conversion.PolygonToRaster("MergeAllBuffered", "raster", "MergeAllBufferedr", "CELL_CENTER", "NONE", SEraster)

### Bring in NLCD

These steps take the 2011 NLCD, pull out all urban classes as 1 and leave everything else as 0, then uses focal statistics\mean to calculate the percent urban in a 270 meter radius (we used 270 instead of 250 since we are using 30 meter pixels).  All pixels that are < 50% urban in a 270 meter radius are considered low urban and are given a value of 1.  All pixels that are > 50% urban in a 279 meter radius are considered high urban and are give a value of 0. 

In [34]:
# Make sure I'm in the workspace to where I am saving the outputs
arcpy.env.workspace = OutWorkspace

In [35]:
# Print the current workspace to make sure I'm in the right spot
print(arcpy.env.workspace)

E:\WORKING\Indicators\Terrestrial\LowUrbanHistoric\2021\LowUrbanHistoric.gdb


In [36]:
# Make a raster with the 4 urban classes as 1 and everything else is 0
with arcpy.EnvManager(extent=SEraster):
    out_raster = arcpy.sa.Con(nlcd, 1, 0, "Value IN (21, 22, 23, 24)"); out_raster.save("NLCDurb")

In [37]:
# calculate focal statistics on above raster. Cacluate mean in a circle neighborhood with a radius of 270 meters. This will give us the
# percent urban in that focal window (because of how we reclassified above)
# This ran before, and it runs now, but now I keep getting an error that I cannot save this to my workspace. Grr.
# I'm going to try to save as a .tif to see if that works
# nope still got the runtime error
# problem solved, my harddrive was running out of space
out_raster = arcpy.sa.FocalStatistics("NLCDurb", "Circle 270 MAP", "MEAN", "DATA", 90); out_raster.save("M270m")

In [38]:
# reclassify so that all pixels that are > 50% urban in a 279 meter radius are considered high urban and are give a value of 0
out_raster = arcpy.sa.Reclassify("M270m", "VALUE", "0 0.500000 1;0.500000 1 0", "DATA"); out_raster.save("M270m50T")

In [39]:
# Mask out areas that are not historic by taking the percent urban raster times the historic areas raster
out_raster = arcpy.sa.Times("MergeAllBufferedr", "M270m50T"); out_raster.save("M270m50TH")

In [40]:
# We decided in the end to change the classification so that 0 = not a historic site, 1 = high urban historic site, 2 = low urban historic site
out_raster = arcpy.sa.Reclassify("M270m50TH", "VALUE", "0 1;1 2;NODATA 0" "DATA"); out_raster.save("LowUrbanHistoric")

In [41]:
# Prep NLCD so we can use it to assign NoData to oceans
out_raster = arcpy.sa.SetNull(nlcd, 1, "\"NLCD_Land\" = \'Unclassified\'"); out_raster.save("NLCDUnNull")

In [42]:
out_raster = arcpy.sa.ExtractByMask("LowUrbanHistoric", "NLCDUnNull"); out_raster.save("indicator")

## because we forgot to use this indicator in nearby subregion zonation runs, we had to go back and clip it down to just the subregion(s) used for zonation. For next year, we want to change this back to using this indicator everywhere it exists.

In [343]:
# pull out subregion(s) where indicator was used
arcpy.analysis.Select(SubR, "sub", "SubRgn IN ('Atlantic Coastal Plain','East Gulf Coastal Plain Sub','Piedmont')")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\4_Indicators\\SouthAtlanticLowUrbanHistoric\\SouthAtlanticLowUrbanHistoric.gdb\\sub'>

In [344]:
# clip raster to subregion(s) where indicator was used
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster):
    out_raster = arcpy.sa.ExtractByMask(indicator1, "sub"); out_raster.save("indicator")

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to SE extent, clip and export to SA extent

In [345]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 2:
        return '2 = Historic place with nearby low-urban buffer'
    elif value == 1:
        return '1 = Historic place with nearby high-urban buffer'
    elif value == 0:
        return '0 = Not in the National Register of Historic Places'    
"""

In [346]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField("indicator", "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'indicator'>

In [351]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 2:
		return 93
	if Value == 1:
		return 161
	else:
		return 255
		
def Reclass2(Value):
	if Value == 2:
		return 64
	if Value == 1:
		return 136
	else:
		return 255
		
def Reclass3(Value):
	if Value == 2:
		return 55
	if Value == 1:
		return 127
	else:
		return 255
		
"""

In [352]:
# calculate Red field
arcpy.management.CalculateField("indicator", "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField("indicator", "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField("indicator", "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'indicator'>

In [353]:
# clip to SE 2022 Blueprint extent
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.ExtractByMask("indicator", SEraster); out_raster.save("SEMask2")

In [354]:
# export as .tif
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster):
    arcpy.management.CopyRaster("SEMask2", SEout, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")